In [ ]:
#importing the pre-requsites
import requests
from datetime import datetime, timedelta
import time
import threading

#function to generate the access token
def get_short_lived_token(client_id, client_secret, redirect_uri, code):
    """
    Exchange an authorization code for a short-lived access token from Facebook's OAuth endpoint.

    Parameters:
    - client_id (str): Your Facebook App's Client ID.
    - client_secret (str): Your Facebook App's Client Secret.
    - redirect_uri (str): The redirect URI registered for your Facebook App.
    - code (str): The authorization code received from the OAuth redirect.

    Returns:
    dict: A dictionary containing the JSON response from Facebook's OAuth endpoint,
          typically including 'access_token', 'token_type', and 'expires_in'.
    """
    token_url = 'https://graph.facebook.com/v10.0/oauth/access_token'
    payload = {
        'client_id': client_id,
        'redirect_uri': redirect_uri,
        'client_secret': client_secret,
        'code': code
    }
    response = requests.get(token_url, params=payload)
    return response.json()

#calaculate the current toke expiry timestamp
def calculate_expiry_time(expires_in):
    """
    Calculate the expiry time based on the current datetime and the given duration in seconds.

    Parameters:
    - expires_in (int): Number of seconds until the token expires.

    Returns:
    datetime: The datetime object representing the calculated expiry time.
    """
    return datetime.now() + timedelta(seconds=expires_in)

#refreshing  token when about to expire
def refresh_short_lived_token(client_id, client_secret, redirect_uri, refresh_token_code):
    """
    Refresh a short-lived access token using a refresh token.

    Parameters:
    - client_id (str): Your Facebook App's Client ID.
    - client_secret (str): Your Facebook App's Client Secret.
    - redirect_uri (str): The redirect URI registered for your Facebook App.
    - refresh_token_code (str): The refresh token or authorization code received from Facebook.

    Returns:
    dict: A dictionary containing the JSON response from Facebook's OAuth endpoint,
          typically including 'access_token', 'token_type', and 'expires_in'.
    """
    token_data = get_short_lived_token(client_id, client_secret, redirect_uri, refresh_token_code)
    return token_data

def schedule_token_refresh(client_id, client_secret, redirect_uri, expiry_time, refresh_token_code, refresh_before_expiry_seconds=3600):
    global short_lived_token, expiry_time
    while True:
        current_time = datetime.now()
        if current_time >= expiry_time - timedelta(seconds=refresh_before_expiry_seconds):
            print("Refreshing token...")
            token_data = refresh_short_lived_token(client_id, client_secret, redirect_uri, refresh_token_code)
            short_lived_token = token_data['access_token']
            expires_in = token_data['expires_in']
            expiry_time = calculate_expiry_time(expires_in)
            print(f"Token refreshed. New expiry time: {expiry_time}")
        time.sleep(600)  # Check every 10 minutes

# Initial Authentication
client_id = '<CLIENT_ID>,123456789012345'
client_secret = '<CLIENT_SECRET>,abcdef1234567890'
redirect_uri = '<REDIRECT_URI>,https://fadsapi.com/oauth_callback'
auth_code = '<AUTH_CODE>,AQDBhO8MnpZPzKNJj8jL8SNtD65fwJsmoI5W4_KSoLUC3zT_d6yNXvrbPcMDi1JnFvLcdW3Q7k_C6bG3oRz0Jh8FqRCJnqyjE9hHgABpKzD8HJn0e3K5Y2C-1Va3V0pJk5sLQIIN3Tl2bM1l-5A5NAAQYJzmE_2yExdy7l-K4ZVLIsBd-9lDFZ2WOB3EC8xH14V0rglK43E8sJlL_X7QvMgkClyh4yyEgYkqRrKZ2C0yQyk9fK14b'

#initial token generation
short_lived_token_data = get_short_lived_token(client_id, client_secret, redirect_uri, auth_code)
short_lived_token = short_lived_token_data['access_token']
expires_in = short_lived_token_data['expires_in']
expiry_time = calculate_expiry_time(expires_in)

# Start the refresh scheduler in a background thread
refresh_token_code = '<AUTH_CODE>'  # This should be the code used for refreshing the token
refresh_thread = threading.Thread(target=schedule_token_refresh, args=(client_id, client_secret, redirect_uri, expiry_time, refresh_token_code))
refresh_thread.start()

api_call_lock = threading.Lock()

#actual api call
def perform_api_call(api_endpoint, params):
    with api_call_lock:
        response = requests.get(api_endpoint, params=params)
        return response.json()

# Example API call
def example_facebook_api_call():
    api_endpoint = 'https://graph.facebook.com/v10.0/<ENDPOINT>'
    params = {'access_token': short_lived_token, 'other_param': 'value'}
    return perform_api_call(api_endpoint, params)
